In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain mcp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.18.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

### MCP Server

In [0]:
server_code = r'''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("dbx-mcp-smoke-test")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Return a + b."""
    return a + b

if __name__ == "__main__":
    # Runs an MCP server over stdio (so a client can connect via pipes)
    mcp.run()
'''

path = "/tmp/mcp_server.py"
with open(path, "w") as f:
    f.write(server_code)

print("Wrote:", path)

Wrote: /tmp/mcp_server.py


In [0]:
import os, sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command=sys.executable,
    args=["/tmp/mcp_server.py"],
    env=os.environ.copy(),
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        tools = await session.list_tools()
        print("Tools:", [t.name for t in tools.tools])

        res = await session.call_tool("add", {"a": 2, "b": 3})
        print("add(2, 3) =>", res.content)


Tools: ['add']
add(2, 3) => [TextContent(type='text', text='5', annotations=None, meta=None)]


### FastMCP

In [0]:
server_code = r'''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("hello-server")

@mcp.tool()
def greet(name: str) -> str:
    """Generates a personalized greeting."""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run()  # stdio transport by default
'''
with open("/tmp/server.py", "w") as f:
    f.write(server_code)
print("Wrote /tmp/server.py")


Wrote /tmp/server.py


### FastMCP over HTTP

In [0]:
import os, sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command=sys.executable,
    args=["/tmp/server.py"],
    env=os.environ.copy(),
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        tools = await session.list_tools()
        print("Tools:", [t.name for t in tools.tools])

        res = await session.call_tool("greet", {"name": "Databricks"})
        print("Result:", res.content)


Tools: ['greet']
Result: [TextContent(type='text', text='Hello, Databricks! Nice to meet you.', annotations=None, meta=None)]


In [0]:
server_code = r'''
from fastmcp import FastMCP

mcp_server = FastMCP()

@mcp_server.tool
def greet(name: str) -> str:
    """Generates a personalized greeting."""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp_server.run(transport="http", host="127.0.0.1", port=8000)
'''
with open("/tmp/server_http.py", "w") as f:
    f.write(server_code)
print("Wrote /tmp/server_http.py")

Wrote /tmp/server_http.py


In [0]:
import subprocess, sys
proc = subprocess.Popen([sys.executable, "/tmp/server_http.py"])
print("server pid:", proc.pid)


server pid: 3839054


In [0]:
proc.terminate()
proc.wait()

1